# Test attack on (Signed) PEP for weakly-self dual codes, using squares of hulls

We generate a pair of codes having length $n$, dimension $k$ and hull dimension $h$, defined over a finite field with $q$ elements.

We first generate the generator matrix $\mathbf G\in\mathbb F_q^{k\times n}$ of a random code with such parameters, then sample a random change of basis $\mathbf S\in GL_k(\mathbb F_q)$ and a random permutation matrix $\mathbf P\in\mathbb F_q^{n\times n}$ and set

$\mathbf G' = \mathbf S\cdot \mathbf G\cdot \mathbf P$

The script optionally tests also the case of Signed PEP. To do this, set $\mathtt{signed}\_\mathtt{PEP} = \mathtt{True}$. In such a case, we sample also a random diagonal matrix $\mathbf D\in\mathbb F_q^{n\times n}$ with coefficients $\pm 1$ and set 

$\mathbf G' = \mathbf S\cdot \mathbf G\cdot \mathbf P\cdot \mathbf D$

In [1]:
#simulate the attack
import time

load('code_utils.sage')
load('ABL_sample_weakly_self_dual.sage')
load("GIP_solver.sage")

# Generate first code

You may do this by importing one of the codes from the folder (if you want to simulate the attack on the ABL scheme), or you can generate a new code. Notice that generation of large weakly-self dual codes takes some time...

In [1]:
import csv

n = 7313
k = 450
q = 8191
h = 27
signed_PEP = True #set to True if you want to test the attack on Signed PEP

id_code = 0

folder_name = "codes_"+str(n)+"_"+str(k)+"_"+str(q)+"_"+str(h)

file_name = "G_"+str(n)+"_"+str(k)+"_"+str(q)+"_"+str(h)+"_"+str(id_code)

Fq = GF(q)
G = matrix(Fq,k,n)
with open(folder_name+"/"+file_name, newline='\n') as csvfile:
    G_reader = csv.reader(csvfile, delimiter=',')
    i = 0
    for row in G_reader:
        for j in range(n):
            G[i,j] = int(row[j])
        i += 1

In [14]:
n = 100
k = 50
q = 127
h = 10
signed_PEP = True #set to True if you want to test the attack on Signed PEP
verbose = False

G = faster_sample_weakly_self_dual_code(q, n, k, h, verbose)

In [12]:
print("Rank(G) = ",rank(G))
print("Hull dim. = ",k-rank(G*G.transpose()))

Rank(G) =  50
Hull dim. =  10


# Sample instance of PEP

In [4]:
Fq = GF(q) #finite field

#sample change of basis
rank_S = 0
while rank_S < k:
    S = random_matrix(Fq, k, k)
    rank_S = rank(S)

#sample random permutation
P_as_list = Permutations(n).random_element()
P = P_as_list.to_matrix().change_ring(Fq)

#sample random diagonal (if signed_PEP = False, this is the identity matrix)
D = identity_matrix(Fq,n)

if signed_PEP:
    for i in range(n):
        val = Set([Fq(1), Fq(-1)]).random_element()
        D[i,i] = val
        
#compute G'
G_prime = S*G*P*D

print("PEP instance generated")

PEP instance generated


In [5]:
####compute hulls
M1 = hull(Fq, G)
M2 = hull(Fq, G_prime)

##square hulls
U1 = square_code(M1) #basis of square of hull for first code
U2 = square_code(M2) #basis of square of hull for second code 

#Compute hulls and print their dimension
#hull_U1 = hull(Fq, U1)
#hull_U2 = hull(Fq, U2)
print("Hull dimension of U1 = ",rank(U1)-rank(U1*U1.transpose()))
print("Hull dimension of U2 = ",rank(U2)-rank(U2*U2.transpose()))

#compute graphs
start = time.time()
A1 = graph_from_generator_matrix(U1)
A2 = graph_from_generator_matrix(U2)
end = time.time()
time_gauss = end - start
print("--> Time for computing graphs = ",time_gauss," s")

#solve GIP
start = time.time()
sol = simple_solve_GIP(Fq, A1, A2)
end = time.time()
time_GIP = end - start
print("--> Time for solving GIP = ",time_GIP," s")

print("Reduction to GIP concluded, total time = ",time_gauss + time_GIP," s")

#Check validity of found permutation
my_P = matrix(Fq, n, n)
for i in range(n):
#    print(sol[i])
    j1 = sol[i][0]
    j2 = sol[i][1]
    my_P[j1, j2] = 1

if my_P == P:
    print("We found the correct permutation!")
else:
    print("Found permutation is wrong, sorry :(")

Hull dimension of U1 =  0
Hull dimension of U2 =  0
--> Time for computing graphs =  0.0016245841979980469  s
--> Time for solving GIP =  0.017229557037353516  s
Reduction to GIP concluded, total time =  0.018854141235351562  s
We found the correct permutation!


#### If considering Signed PEP, try also finding scalar coefficients

In [7]:
time_in = time.time()

##find D
perm_G = G*my_P

#set up matrix
A = matrix(Fq,k*(n-k),n)
num = 0

H_prime = codes.LinearCode(G_prime).parity_check_matrix()

for i in range(k):
    for j in range(n-k):
        for ell in range(n):
            A[num,ell] = perm_G[i,ell]*H_prime[j,ell]
        num += 1

#find kernel of A
solution_space = A.right_kernel()
B = solution_space.basis()
print("Dimension of solution space = ",len(B))

b = B[0]

b1 = b[0]^-1 * b
b2 = -b1

#see if b1 is ok
ok1 = 1
i = 0
while (ok1 == 1)&(i<n):
    if b1[i]==D[i,i]:
        i += 1
    else:
        ok1 = 0

#see if b2 is ok
ok2 = 1
i = 0
while (ok2 == 1)&(i<n):
    if b2[i]==D[i,i]:
        i += 1
    else:
        ok2 = 0

#check validity of solution
if (ok1 == 1)|(ok2 == 1):
    print("We retrieved also the scalar coefficients!")
else:
    print("Something went wrong!")
    
time_fin = time.time()

print("Tot. time = ",time_fin - time_in)

Dimension of solution space =  1
We retrieved also the scalar coefficients!
Tot. time =  0.09004545211791992


##### Try finding $D$ using less equations

If the matrix $\mathbf A$ cannot be stored, use less equations

In [10]:
time_in = time.time()

perm_G = G*my_P

#sample change of basis
rank_S1 = 0
while rank_S1 < k:
    S1 = random_matrix(Fq, k, k)
    rank_S1 = rank(S1)
rank_S2 = 0
while rank_S2 < n-k:
    S2 = random_matrix(Fq, n-k, n-k)
    rank_S2 = rank(S2)

perm_G = S1*perm_G
H_prime = S2*codes.LinearCode(G_prime).parity_check_matrix()

#set up matrix
num_G = ceil(sqrt(n*k/(n-k)))+5
num_H = ceil(sqrt(n*(n-k)/k))+5
A = matrix(Fq,num_G*num_H,n)

num = 0

pos_G = Combinations(k,num_G).random_element()
for i in pos_G:
    pos_H = Combinations(n-k,num_H).random_element()
    for j in pos_H:
        for ell in range(n):
            A[num,ell] = perm_G[i,ell]*H_prime[j,ell]
        num += 1

#find kernel of A
solution_space = A.right_kernel()
B = solution_space.basis()
print("Dimension of solution space = ",len(B))

b = B[0]

b1 = b[0]^-1 * b
b2 = -b1

#see if b1 is ok
ok1 = 1
i = 0
while (ok1 == 1)&(i<n):
    if b1[i]==D[i,i]:
        i += 1
    else:
        ok1 = 0

#see if b2 is ok
ok2 = 1
i = 0
while (ok2 == 1)&(i<n):
    if b2[i]==D[i,i]:
        i += 1
    else:
        ok2 = 0

#check validity of solution
if (ok1 == 1)|(ok2 == 1):
    print("We retrieved also the scalar coefficients!")
else:
    print("Something went wrong!")
    
time_fin = time.time()

print("Tot. time = ",time_fin - time_in)

Dimension of solution space =  1
We retrieved also the scalar coefficients!
Tot. time =  0.019183635711669922
